## Particion de dataset para carga incremental
STORY:
Los archivos son muy grandes por lo que se debe hacer la carga por partes en la base de datos Mysql, para lo
cual se prepara este script, adicionalmente para probar el escrip de carga incremental se modifica la data creando 
valores repetidos y archivos cuyo contenido no coincide con el formato esperado. 

### 0.Condiciones de incio

In [1]:
#Librerias a utilizar
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [2]:
#Direccion incial donde estan archivos de entrada y donde se almacenaran archivos salida
path_incial = 'C:\\Github\\DataSet_YELP\\'
path_final = 'C:\\Github\\DataSet_YELP\\pruebas_incremental\\'

### 1.Creacion de datos prueba para review

In [4]:
#Carga de archivo review en tabla review
nombre_archivo = 'review.json'
nombre = nombre_archivo.split('.')[0]

In [5]:
cont = 0
cont_max = 34
Tamano = 200000
review = pd.DataFrame()
for chunk in pd.read_json(path_incial+nombre_archivo,chunksize=Tamano, lines=True):
    # Visualizacion de avance
    clear_output(wait=True)
    print('Completado Etapa 1: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
        
    #Armado de archivo
    if cont == 0:
        chunk.to_json(path_final+nombre+'_inicial.json',orient="records",lines=True)
    elif cont < 30:
        chunk.to_json(path_final+nombre+'_incremental_'+str(cont)+'.json',orient="records",lines=True)
    else:
        df = pd.read_json(path_final+nombre+'_incremental_'+str(np.random.randint(1,30))+'.json',lines=True)
        review = pd.concat([df.sample(20000), chunk], ignore_index=True)
        review.to_json(path_final+nombre+'_incremental_con_repeticiones'+str(cont)+'.json',orient="records",lines=True)
    cont = cont + 1
    
#tiempo aprox = 7 min

Completado Etapa 1: 34/34  100.0%


In [8]:
df = pd.DataFrame(np.random.randn(1000, 4), columns=list('ABCD'))
df.to_json(path_final+nombre+'_aleatorio1.json',lines=True,orient="records")

columnas = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny','cool', 'text', 'date']
df = pd.DataFrame(np.random.randn(1000, len(columnas)), columns=columnas)
df.to_json(path_final+nombre+'_aleatorio2.json',lines=True,orient="records")

Creado :C:\Github\DataSet_YELP\pruebas_incremental\review_aleatorio2.json


### 2.Creacion datos de prueba bussines y user

In [5]:
review_inicial = pd.read_json(path_final+nombre+'_inicial.json',lines=True)
user_id = pd.DataFrame(data = review_inicial['user_id'].unique(),columns = ['user_id'])
bussines_id = pd.DataFrame(data = review_inicial['business_id'].unique(),columns = ['business_id'])

#### 2.1 Creacion de datos de prueba user

In [6]:
nombre_archivo = 'user.json'
nombre = nombre_archivo.split('.')[0]

In [7]:
user_inicial = pd.DataFrame()
cont = 0
cont_max = 9
Tamano = 200000
for chunk in pd.read_json(path_incial+nombre_archivo,chunksize=Tamano, lines=True):
     # Visualizacion de avance
    clear_output(wait=True)
    print('Completado : '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    #Se verifica que filas del chunk de la tabla user tienen codigo 'user_id' en review_inicial
    df_chunk_user_id =  pd.merge(user_id,chunk,on = 'user_id',how='outer',indicator=True)

    #Se agrega a la tabla "user_inicial" cualquier fila que este presente en review_inicial
    user_inicial_chunk = df_chunk_user_id[df_chunk_user_id['_merge']=='both'].drop(['_merge'], axis=1)
    user_inicial = pd.concat([user_inicial, user_inicial_chunk], ignore_index=True) 
    
    #Nuevas filas de user se cargaran de forma incremental
    user_incremental_chunk = df_chunk_user_id[df_chunk_user_id['_merge']=='right_only'].drop(['_merge'], axis=1)
    if cont < 8:
        user_incremental_chunk.to_json(path_final+nombre+'_incremental_'+str(cont)+'.json',orient="records",lines=True)
    else:
        user_incremental_repetidos =  pd.concat([user_inicial.sample(user_inicial.shape[0]//5), user_incremental_chunk], ignore_index=True) 
        user_incremental_repetidos.to_json(path_final+nombre+'_incremental_con_repeticiones'+str(cont)+'.json',orient="records",lines=True) 
        pass
    cont = cont + 1

user_inicial.to_json(path_final+nombre+'_inicial.json',orient="records",lines=True)

#tiempo aprox = 4 min

Completado : 9/9  100.0%


In [10]:
df = pd.DataFrame(np.random.randn(1000, 4), columns=list('ABCD'))
df.to_json(path_final+nombre+'_aleatorio1.json',lines=True,orient="records")

columnas = ['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos']
df = pd.DataFrame(np.random.randn(1000, len(columnas)), columns=columnas)
df.to_json(path_final+nombre+'_aleatorio2.json',lines=True,orient="records")

#### 2.2 Creacion de datos de prueba bussines

In [37]:
nombre_archivo = 'business.json'
nombre = nombre_archivo.split('.')[0]

In [39]:
business = pd.read_json(path_incial+nombre_archivo,lines=True)

#Se verifica que filas del chunk de la tabla user tienen codigo 'user_id' en review_inicial
df_business =  pd.merge(bussines_id,business,on = 'business_id',how='outer',indicator=True)

business_inicial = df_business[df_business['_merge']=='both'].drop(['_merge'], axis=1)
business_inicial.to_json(path_final+nombre+'_inicial.json',orient="records",lines=True)


business_incremental = df_business[df_business['_merge']=='right_only'].drop(['_merge'], axis=1)

business_incremental1 = business_incremental.iloc[0:business_incremental.shape[0]//2,:]
business_incremental1.to_json(path_final+nombre+'_incremental.json',orient="records",lines=True)

business_incremental2 = business_incremental.iloc[business_incremental.shape[0]//2+1:,:]
repetidos  = business_inicial.sample(business_inicial.shape[0]//4)
repetidos  = pd.concat([business_incremental2, repetidos], ignore_index=True)
repetidos.to_json(path_final+nombre+'_incremental_con_repeticiones.json',orient="records",lines=True)


In [40]:
df = pd.DataFrame(np.random.randn(1000, 4), columns=list('ABCD'))
df.to_json(path_final+nombre+'_aleatorio1.json',lines=True,orient="records")

columnas = ['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours']
df = pd.DataFrame(np.random.randn(1000, len(columnas)), columns=columnas)
df.to_json(path_final+nombre+'_aleatorio2.json',lines=True,orient="records")